In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import itertools
import threading
import time
import sys
from termcolor import colored

In [336]:
cities = ['Dublin','Amsterdam','Madrid','Paris','Berlin','Milan','London','Barcelona','Frankfurt','Lisbon']
#Glassdoor
da_table=pd.read_csv('glassdoor_role_table_unique2roles.csv')
#Numbeo
numbeo_summary_table = pd.read_csv('numbeo_table_summary.csv')
numbeo_item_table =  pd.read_csv('numbeo_table_priceitems.csv')
numbeo_item_table.rename(columns = {'Country':'City'}, inplace = True)
#Nomad list
nomad_list =pd.read_csv('df_nomad_list.csv')
#Columns adjugement
columnsok = []
for x in nomad_list.columns[1:]:
    b = re.findall(r'(?<=\s).*',x)
    columnsok.append(b)
columnsok
col = []

for x in range(0,80):
     col.append(columnsok[x][0])
col.insert(0,"City")
col
nomad_list.set_axis(col,axis=1,inplace=True)
nomad_list

City                    Total score Quality of life score  \
0     Dublin  3.02/5 (Rank #774)239 reviews                  Good   
1  Amsterdam   3.79/5 (Rank #78)266 reviews                  Good   
2     Madrid   3.88/5 (Rank #60)257 reviews                  Good   
3      Paris  3.63/5 (Rank #127)262 reviews                  Good   
4     Berlin   4.27/5 (Rank #20)265 reviews                  Good   
5      Milan  2.96/5 (Rank #877)246 reviews                  Good   
6     London   3.84/5 (Rank #71)257 reviews                  Good   
7  Barcelona   3.88/5 (Rank #59)263 reviews                  Good   
8  Frankfurt  3.59/5 (Rank #136)240 reviews                  Good   
9     Lisbon    4.64/5 (Rank #3)256 reviews                  Good   

  Family score                              Cost              Internet    Fun  \
0         Good  😳 Way too expensive: $5,038 / mo  🏎 Fast: 47Mbps (avg)   Okay   
1         Good  😳 Way too expensive: $5,828 / mo  🏎 Fast: 14Mbps (avg)   Good   
2         Good             🧐 Pricey: $3,259 / mo  🏎 Fast: 39Mbps (avg)  Great   
3         Good      🧐 Too expensive: $4,261 / mo  🏎 Fast: 24Mbps (avg)   Good   
4         Good             🧐 Pricey: $3,534 / mo  🏎 Fast: 24Mbps (avg)  Great   
5         Okay      🧐 Too expensive: $4,733 / mo  🏎 Fast: 13Mbps (avg)   Okay   
6         Good      🧐 Too expensive: $4,766 / mo  🏎 Fast: 15Mbps (avg)  Great   
7         Good      🧐 Too expensive: $4,125 / mo  🏎 Fast: 39Mbps (avg)  Great   
8         Good               🙂 Okay: $2,813 / mo  🏎 Fast: 15Mbps (avg)   Good   
9         Good         😙 Affordable: $2,452 / mo  🏎 Fast: 35Mbps (avg)  Great   

                         Temperature (now) Humidity (now)   Air quality (now)  \
0  🥶 A bit cold: 16°C60°F (feels 16°C61°F)    💦 Damp: 86%  🌱 Great: 12 US AQI   
1    🥶 Too cold: 14°C58°F (feels 14°C57°F)   😊 Comfy: 70%  🌱 Great: 32 US AQI   
2     🌞 Perfect: 22°C72°F (feels 22°C72°F)   😊 Comfy: 52%  🌱 Great: 42 US AQI   
3  🥶 A bit cold: 17°C62°F (feels 18°C64°F)   😊 Comfy: 58%   🌱 Good: 60 US AQI   
4  🥶 A bit cold: 16°C61°F (feels 16°C61°F)   😊 Comfy: 57%  🌱 Great: 50 US AQI   
5        🌞 Nice: 21°C70°F (feels 21°C70°F)   😊 Comfy: 61%  🌱 Great: 46 US AQI   
6    🥶 Too cold: 13°C55°F (feels 13°C55°F)   😊 Comfy: 69%   🌱 Good: 53 US AQI   
7        🌞 Nice: 20°C69°F (feels 21°C70°F)   😊 Comfy: 78%  🌱 Great: 15 US AQI   
8        🌞 Nice: 18°C65°F (feels 18°C64°F)   😊 Comfy: 70%  🌱 Great: 30 US AQI   
9        🌞 Nice: 19°C66°F (feels 19°C66°F)   😊 Comfy: 74%  🌱 Great: 40 US AQI   

   ... Hotel (median price).1 Airbnb (median from 180 listings)  \
0  ...           $123 / night                    $8,574 / month   
1  ...           €166 / night                    €8,293 / month   
2  ...            €90 / night                    €3,557 / month   
3  ...           €140 / night                    €3,642 / month   
4  ...            €80 / night                    €4,768 / month   
5  ...           $112 / night                    $5,918 / month   
6  ...           €139 / night                    €4,438 / month   
7  ...           €141 / night                    €5,050 / month   
8  ...            $73 / night                    $2,670 / month   
9  ...            $65 / night                    $3,532 / month   

  Airbnb (median price) Dinner Coca-Cola (0.3L) Beer (0.5L) Coffee  \
0          $281 / night    $11               $2          $5     $4   
1          €272 / night    €13            €2.25       €5.00  €3.75   
2          €117 / night    €11            €1.84       €3.01  €1.50   
3          €119 / night    €13            €2.50       €6.00  €2.00   
4          €156 / night  €9.50            €2.00       €3.00  €3.00   
5          $194 / night    $17               $3          $6     $2   
6          €146 / night    €13            €1.59       €4.79  €3.20   
7          €166 / night    €10            €2.00       €3.01  €1.90   
8           $88 / night     $8               $2          $4     $3   
9          $116 / night   

In [256]:
df_3 = numbeo_item_table[(numbeo_item_table['City'] == city_orig) | (numbeo_item_table['City'] == city_dest)].reset_index()
df_3

level_0  Unnamed: 0  index  Meal, Inexpensive Restaurant   \
0        0           0      0                          15.00   
1        2           2      0                          12.95   

   Meal for 2 People, Mid-range Restaurant, Three-course   \
0                                               70.0        
1                                               60.0        

   McMeal at McDonalds (or Equivalent Combo Meal)   \
0                                              9.0   
1                                              8.0   

   Domestic Beer (0.5 liter draught)   Imported Beer (0.33 liter bottle)   \
0                                 5.7                                2.92   
1                                 3.0                                1.43   

   Cappuccino (regular)   Coke/Pepsi (0.33 liter bottle)   ...  \
0                   3.62                             2.01  ...   
1                   2.05                             2.16  ...   

   1 Pair of Men Leather Business Shoes   \
0                                  94.00   
1                                  94.23   

   Apartment (1 bedroom) in City Centre   \
0                               1,817.16   
1                               1,029.33   

   Apartment (1 bedroom) Outside of Centre   \
0                                  1,473.27   
1                                    785.78   

   Apartment (3 bedrooms) in City Centre   \
0                                2,978.57   
1                                1,786.00   

   Apartment (3 bedrooms) Outside of Centre   \
0                                   2,342.86   
1                                   1,261.15   

   Price per Square Meter to Buy Apartment in City Centre   \
0                                           7,677.30         
1                                           4,852.96         

   Price per Square Meter to Buy Apartment Outside of Centre   \
0                                           4,472.13            
1                                           3,468.08            

   Average Monthly Net Salary (After Tax)   \
0                                 3,089.63   
1                                 1,836.54   

   Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate   \
0                                               3.16                             
1                                               1.99                             

     City  
0  Dublin  
1  Madrid  

[2 rows x 58 columns]

In [252]:
df_3.columns

Index(['level_0', 'Unnamed: 0', 'index', 'Meal, Inexpensive Restaurant ',
       'Meal for 2 People, Mid-range Restaurant, Three-course ',
       'McMeal at McDonalds (or Equivalent Combo Meal) ',
       'Domestic Beer (0.5 liter draught) ',
       'Imported Beer (0.33 liter bottle) ', 'Cappuccino (regular) ',
       'Coke/Pepsi (0.33 liter bottle) ', 'Water (0.33 liter bottle)  ',
       'Milk (regular), (1 liter) ', 'Loaf of Fresh White Bread (500g) ',
       'Rice (white), (1kg) ', 'Eggs (regular) (12) ', 'Local Cheese (1kg) ',
       'Chicken Fillets (1kg) ',
       'Beef Round (1kg) (or Equivalent Back Leg Red Meat) ', 'Apples (1kg) ',
       'Banana (1kg) ', 'Oranges (1kg) ', 'Tomato (1kg) ', 'Potato (1kg) ',
       'Onion (1kg) ', 'Lettuce (1 head) ', 'Water (1.5 liter bottle) ',
       'Bottle of Wine (Mid-Range) ', 'Domestic Beer (0.5 liter bottle) ',
       'Cigarettes 20 Pack (Marlboro) ', 'One-way Ticket (Local Transport) ',
       'Monthly Pass (Regular Price) ', 'Taxi Sta

In [337]:
nomad_list.columns

Index(['City', 'Total score', 'Quality of life score', 'Family score', 'Cost',
       'Internet', 'Fun', 'Temperature (now)', 'Humidity (now)',
       'Air quality (now)', 'Air quality (annual)', 'Safety', 'Lack of crime*',
       'Lack of racism*', 'Education level*', 'Income level*',
       'Liked by members', 'English speaking*', 'People density',
       'Walkability', 'Peace (no pol. conflict)', 'Traffic safety*',
       'Hospitals', 'Happiness*', 'Nightlife', 'Free WiFi in city',
       'Places to work from', 'A/C or heating', 'Friendly to foreigners',
       'Freedom of speech*', 'Female friendly', 'LGBTQ+ friendly',
       'Startup Score', 'Continent', 'Country', 'Average trip length',
       'Internet speed (avg)', 'Weather (now)', 'Air quality (now).1',
       'Air quality (annual avg)', 'Power', 'Best neighborhood to stay',
       'Upcoming neighborhood', 'Best taxi app*', 'Travel medical insurance',
       'Best wireless carrier', 'Suggested ATM take out:', 'Tipping',
      

In [345]:
df_5 = nomad_list[(nomad_list['City'] == city_orig) | (nomad_list['City'] == city_dest)].reset_index()
#NightLife
nightlife_dest = df_5['Nightlife'][df_5['City'] == city_dest][1]
print(nightlife_dest)
nightlife_org= df_5['Nightlife'][df_5['City'] == city_orig][0]
print(nightlife_org)

Great
Okay


In [348]:
#Fun
fun_dest = df_5['Fun'][df_5['City'] == city_dest][1]
print(fun_dest)
fun_org= df_5['Fun'][df_5['City'] == city_orig][0]
print(fun_org)

Great
Okay


In [349]:
#Fender ratio
gender_dest = df_5['Gender ratio (overall)'][df_5['City'] == city_dest][1]
print(gender_dest)
gender_org= df_5['Gender ratio (overall)'][df_5['City'] == city_orig][0]
print(gender_org)

👨 43% 👱‍♀️ 57%
👨 47% 👱‍♀️ 53%


In [184]:
print('Average Monthly Net Salary (After Tax): ')
print('Etimated monthly costs single person without rent: ')
print('Apartment (1 bedroom) in City Centre: ')

Average Monthly Net Salary (After Tax): 
stimated monthly costs single person without rent: 
Apartment (1 bedroom) in City Centre: 


In [170]:
done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rloading ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!')

t = threading.Thread(target=animate)
t.start()

#long process here
time.sleep(10)
done = True
print("\n")

loading \

Done!

In [368]:
#MAin program

print('\033[1m','....................................Welcome to Choose&move....................................''\033[0m')
#Scraping proccess simulation
#TBD
done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rScraping last updates     ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    print('\rDone:Last update:Updated 3 May 2022      ')

t = threading.Thread(target=animate)
t.start()

#long process here
time.sleep(5)
done = True
print("\n")


#long process here
time.sleep(2)
done = True
#program
print('    ')
print('\033[94m','Information available for:','\033[0m')
[print('\033[0;32m',city,'\033[0m') for city in cities]
print('    ')
city_orig = input('Introduce current city: ')
city_dest = input('Introduce city to compare: ')
print('    ')
print('\033[0;34m','Information available for:','\033[0m')
print('\033[0;32m','Junior Data Analyst''\033[0m')
print('\033[0;32m','Junior Web Developer''\033[0m')
print('    ')
role= input('Role position to analyze: ',)
print('    ')
#filter table by position and role
df_1 = da_table[(da_table['city'] == city_orig) | (da_table['city'] == city_dest)]
df_1= df_1[df_1.position.str.contains(f'{role}', flags = re.IGNORECASE, regex = True, na = False)].reset_index()

print('\033[0;34m' ,'Salary reference:','\033[0m')

#Avg salary origin  and to comparecity
avg_org_city = df_1["avgbasepay"][df_1['city'] == city_orig][0]
avg_org_city
avg_dest_city = df_1["avgbasepay"][df_1['city'] == city_dest][1]
avg_dest_city

print(f'Average salary in {city_orig} : {avg_org_city} | {city_dest} : {avg_dest_city}  ')
print('    ')
print('\033[0;34m' ,'Range Salaries:','\033[0m')
#Range Salaries table
df_1.rename(columns = {'city':'City', 'minpay':'Low','avgpay':'Average', 'maxpay':'High'}, inplace = True)
#Changing colum names
display(df_1[['City','Low','Average','High']])
#Cost of life part
print('    ')
print('\033[0;34m' ,'Cost of Life:','\033[0m')

#Estimated monthly costs single person without rent
df_2 = numbeo_summary_table[(numbeo_summary_table['City'] == city_orig) | (numbeo_summary_table['City'] == city_dest)].reset_index()
cost_cityorig = df_2["A single person estimated monthly costs are without rent."][df_2['City'] == city_orig][0]
#cost_cityorig
cost_citydest = df_2["A single person estimated monthly costs are without rent."][df_2['City'] == city_dest][1]
#cost_citydest

#Apartment (1 bedroom) in City Centre
df_3 = numbeo_item_table[(numbeo_item_table['City'] == city_orig) | (numbeo_item_table['City'] == city_dest)].reset_index()
apartment1bed_cityorig = df_3['Apartment (1 bedroom) in City Centre '][df_3['City'] == city_orig][0]
#apartment1bed_cityorig
apartment1bed_citydest = df_3['Apartment (1 bedroom) in City Centre '][df_3['City'] == city_dest][1]
#apartment1bed_citydest

#Apartment (1 bedroom) in City Centre
df_4 = numbeo_item_table[(numbeo_item_table['City'] == city_orig) | (numbeo_item_table['City'] == city_dest)].reset_index()
sal_cityorig = df_4['Average Monthly Net Salary (After Tax) '][df_4['City'] == city_orig][0]
#sal_cityorig
sal_citydest = df_4['Average Monthly Net Salary (After Tax) '][df_4['City'] == city_dest][1]
#sal_citydest



print(f'Average Monthly Net Salary (After Tax):  {city_orig}:','€', sal_cityorig,' | ',f'{city_dest}:','€',sal_citydest)
print(f'Estimated monthly costs single person without rent:  {city_orig}:','€', cost_cityorig,' | ',f'{city_dest}:','€',cost_citydest)
print(f'Apartment (1 bedroom) in City Centre:  {city_orig}:','€', apartment1bed_cityorig,' | ',f'{city_dest}:','€',apartment1bed_citydest)
print(' ')

print('\033[0;34m' ,'Quality of life:','\033[0m')

df_5 = nomad_list[(nomad_list['City'] == city_orig) | (nomad_list['City'] == city_dest)].reset_index()
#NightLife
nightlife_dest = df_5['Nightlife'][df_5['City'] == city_dest][1]
nightlife_org= df_5['Nightlife'][df_5['City'] == city_orig][0]
#Fun
fun_dest = df_5['Fun'][df_5['City'] == city_dest][1]
fun_org= df_5['Fun'][df_5['City'] == city_orig][0]
#Fender ratio
gender_dest = df_5['Gender ratio (overall)'][df_5['City'] == city_dest][1]
gender_org= df_5['Gender ratio (overall)'][df_5['City'] == city_orig][0]

print(f'Night Life in {city_orig}:',nightlife_org,f' in {city_dest}:',nightlife_dest)
print(f'Fun in {city_orig}:',fun_org,f' in {city_dest}:',fun_dest)
print(f'Gender Ratio in {city_orig}:',gender_org,f' in {city_dest}:',gender_dest)



 ....................................Welcome to Choose&move....................................
Scraping last updates     /

Done:Last update:Updated 3 May 2022      
    
 Information available for: 
 Dublin 
 Amsterdam 
 Madrid 
 Paris 
 Berlin 
 Milan 
 London 
 Barcelona 
 Frankfurt 
 Lisbon 
    
Introduce current city: Dublin
Introduce city to compare: Madrid
    
 Information available for: 
 Junior Data Analyst
 Junior Web Developer
    
Role position to analyze: Junior Data Analyst
    
 Salary reference: 
Average salary in Dublin : €33,139 | Madrid : €26,404  
    
 Range Salaries: 


City   Low Average  High
0  Dublin  €29K    €33K  €40K
1  Madrid  €20K    €26K  €32K

    
 Cost of Life: 
Average Monthly Net Salary (After Tax):  Dublin: € 3,089.63  |  Madrid: € 1,836.54
Estimated monthly costs single person without rent:  Dublin: € 916.05  |  Madrid: € 725.61
Apartment (1 bedroom) in City Centre:  Dublin: € 1,817.16  |  Madrid: € 1,029.33
 
 Quality of life: 
Night Life in Dublin: Okay  in Madrid: Great
Fun in Dublin: Okay  in Madrid: Great
Gender Ratio in Dublin: 👨 47% 👱‍♀️ 53%  in Madrid: 👨 43% 👱‍♀️ 57%


In [360]:
print('\033[1m''Titulo')

Titulo


In [270]:
#nomad_list.columns

Index(['City', '⭐️ Total score', '👍 Quality of life score', '👶 Family score',
       '💵 Cost', '📡 Internet', '😝 Fun', '⛅️ Temperature (now)',
       '💦 Humidity (now)', '💨 Air quality (now)', '💨 Air quality (annual)',
       '👌 Safety', '👮‍♀️ Lack of crime*', '🤝 Lack of racism*',
       '🎓 Education level*', '💰 Income level*', '❤️ Liked by members',
       '🙊 English speaking*', '😤 People density', '🚶 Walkability',
       '✌️ Peace (no pol. conflict)', '🚦 Traffic safety*', '🏥 Hospitals',
       '😄 Happiness*', '🍸 Nightlife', '📶 Free WiFi in city',
       '🖥 Places to work from', '❄️ A/C or heating',
       '😁 Friendly to foreigners', '🗯 Freedom of speech*', '👩 Female friendly',
       '🌈 LGBTQ+ friendly', '🎅 Startup Score', '🌍 Continent', '🚩 Country',
       '⏱ Average trip length', '📡 Internet speed (avg)', '⛅️ Weather (now)',
       '💨 Air quality (now).1', '💨 Air quality (annual avg)', '🔌 Power',
       '🧔 Best neighborhood to stay', '🚀 Upcoming neighborhood',
       '🚕 Best taxi ap

In [334]:
#Columns adjugement
columnsok = []
for x in nomad_list.columns[1:]:
    b = re.findall(r'(?<=\s).*',x)
    columnsok.append(b)
columnsok
col = []

for x in range(0,80):
     col.append(columnsok[x][0])
col.insert(0,"City")
col

['City',
 'Total score',
 'Quality of life score',
 'Family score',
 'Cost',
 'Internet',
 'Fun',
 'Temperature (now)',
 'Humidity (now)',
 'Air quality (now)',
 'Air quality (annual)',
 'Safety',
 'Lack of crime*',
 'Lack of racism*',
 'Education level*',
 'Income level*',
 'Liked by members',
 'English speaking*',
 'People density',
 'Walkability',
 'Peace (no pol. conflict)',
 'Traffic safety*',
 'Hospitals',
 'Happiness*',
 'Nightlife',
 'Free WiFi in city',
 'Places to work from',
 'A/C or heating',
 'Friendly to foreigners',
 'Freedom of speech*',
 'Female friendly',
 'LGBTQ+ friendly',
 'Startup Score',
 'Continent',
 'Country',
 'Average trip length',
 'Internet speed (avg)',
 'Weather (now)',
 'Air quality (now).1',
 'Air quality (annual avg)',
 'Power',
 'Best neighborhood to stay',
 'Upcoming neighborhood',
 'Best taxi app*',
 'Travel medical insurance',
 'Best wireless carrier',
 'Suggested ATM take out:',
 'Tipping',
 'Cashless',
 'Tap water',
 'Return rate',
 'Visitors pe

In [ ]:
avg_org_city = df_1["avgbasepay"][df_1['city'] == city_orig][0]
avg_org_city
#avg_dest_city = df_1["avgbasepay"][df_1['city'] == city_dest][0]
#avg_dest_city
print(f'Average salary in {city_orig} : {avg_org_city} |')
#print(f'Average salary in {city_orig} : {avg_org_city} | {city_dest} : {avg_dest_city}  ')
print('    ')

In [156]:
avg_dest_city = df_1["avgbasepay"][df_1['city'] == city_dest]
avg_dest_city

1    €24,256
Name: avgbasepay, dtype: object

In [112]:
#df_1["minpay"][0]

In [104]:
avg_org_city = df_1["avgbasepay"][df_1['city'] == city_orig][0]
avg_org_city
print("Hola" , avg_org_city)

Hola €33,139


In [77]:
df_1 = da_table[(da_table['city'] == city_orig) | (da_table['city'] == city_dest)]
df_1

Unnamed: 0  index    city  \
0            0      0  Dublin   
2            2      0  Madrid   
10          10      0  Dublin   
12          12      0  Madrid   

                                             position          updatedate  \
0        Junior Web Developer Salaries in Dublin Area  Updated 3 May 2022   
2   Junior Web Developer Salaries in Madrid, Spain...  Updated 3 May 2022   
10        Junior Data Analyst Salaries in Dublin Area  Updated 2 May 2022   
12  Junior Data Analyst Salaries in Madrid, Spain ...  Updated 2 May 2022   

   avgbasepay minpay avgpay maxpay  \
0     €30,972   €26K   €31K   €36K   
2     €23,214   €15K   €23K   €42K   
10    €33,139   €29K   €33K   €40K   
12    €26,404   €20K   €26K   €32K   

                                         positioninfo  
0   How much does a Junior Web Developer make in D...  
2   The average salary for Junior Web Developer is...  
10  How much does a Junior Data Analyst make in Du...  
12  The average salary for Junior Data Analyst is ...

In [ ]:
da_table[da_table['city'] == 'Dublin']

In [50]:

df_1[df_1.position.str.contains(f'{role}', flags = re.IGNORECASE, regex = True, na = False)]

Unnamed: 0  index    city  \
10          10      0  Dublin   
12          12      0  Madrid   

                                             position          updatedate  \
10        Junior Data Analyst Salaries in Dublin Area  Updated 2 May 2022   
12  Junior Data Analyst Salaries in Madrid, Spain ...  Updated 2 May 2022   

   avgbasepay minpay avgpay maxpay  \
10    €33,139   €29K   €33K   €40K   
12    €26,404   €20K   €26K   €32K   

                                         positioninfo  
10  How much does a Junior Data Analyst make in Du...  
12  The average salary for Junior Data Analyst is ...

In [53]:
df_1 = da_table[(da_table['city'] == city_orig) | (da_table['city'] == city_dest)]
df_1= df_1[df_1.position.str.contains(f'{role}', flags = re.IGNORECASE, regex = True, na = False)]
df_1

Unnamed: 0  index    city  \
10          10      0  Dublin   
12          12      0  Madrid   

                                             position          updatedate  \
10        Junior Data Analyst Salaries in Dublin Area  Updated 2 May 2022   
12  Junior Data Analyst Salaries in Madrid, Spain ...  Updated 2 May 2022   

   avgbasepay minpay avgpay maxpay  \
10    €33,139   €29K   €33K   €40K   
12    €26,404   €20K   €26K   €32K   

                                         positioninfo  
10  How much does a Junior Data Analyst make in Du...  
12  The average salary for Junior Data Analyst is ...

In [125]:
import itertools
import threading
import time
import sys

done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rScraping last updates ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!')

t = threading.Thread(target=animate)
t.start()

#long process here
time.sleep(5)
done = True

Done!ing last updates /

In [133]:
import itertools
import threading
import time
import sys

done = False
#here is the animation
def animate():
    for c in itertools.cycle(['|', '/', '-', '\\']):
        if done:
            break
        sys.stdout.write('\rScraping last updates ' + c)
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\rDone!                             ')

t = threading.Thread(target=animate)
t.start()

#long process here
time.sleep(2)
done = True

Done!                             

In [142]:
for j in range(1,5):
    print('\rwaiting : '+str(j), end='')

waiting : 4

In [143]:
_last_print_len = 0
def reprint(msg, finish=False):
    global _last_print_len
    
    # Ovewrites line with spaces.
    print(' '*_last_print_len, end='\r')
    
    if finish:
        end = '\n'
        # If we're finishing the line, we won't need to overwrite it in the next print.
        _last_print_len = 0
    else:
        end = '\r'
        # Store len for the next print.
        _last_print_len = len(msg)
    
    # Printing message.
    print(msg, end=end)

In [146]:
reprint('hola')

NameError: name 'hola' is not defined